In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import precision_score as precision, recall_score as recall, accuracy_score as accuracy
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import os


from graph import *
from graph_characteristic import *
from graph_simulate_statistics import *

# **Нагенерируем датасетиков для разных размеров выборок**

In [4]:
df_25 = generate_dataset(gen_gamma_points, gen_weibull_points, dataset_size=10000, num_vertex=25)
df_25.to_csv(os.path.join('..', 'data', 'df_25_vert_Student2.csv'), index=False)

100%|██████████| 5000/5000 [00:15<00:00, 327.55it/s] 


In [5]:
df_100 = generate_dataset(gen_gamma_points, gen_weibull_points, dataset_size=10000, num_vertex=100)
df_100.to_csv(os.path.join('..', 'data', 'df_100_vert_Student2.csv'), index=False)

100%|██████████| 5000/5000 [06:17<00:00, 13.25it/s]


In [6]:
df_500 = generate_dataset(gen_gamma_points, gen_weibull_points, dataset_size=10000, num_vertex=500)
df_500.to_csv(os.path.join('..', 'data', 'df_500_vert_Student2.csv'), index=False)

100%|██████████| 5000/5000 [8:26:38<00:00,  6.08s/it]    


# **Поехали обучать**

In [7]:
df_25 = pd.read_csv(os.path.join('..', 'data', 'df_25_vert_Student2.csv'))
df_100 = pd.read_csv(os.path.join('..', 'data', 'df_100_vert_Student2.csv'))
df_500 = pd.read_csv(os.path.join('..', 'data', 'df_500_vert_Student2.csv'))

## **Базовые линейные модели**

In [8]:
for n, df in zip([25, 100, 500], [df_25, df_100, df_500]):
    X, y = df.drop('distribution', axis = 1), df['distribution']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=239, stratify=y)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = LogisticRegression(max_iter=1000, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    y_pred_logistic = model.predict(X_test_scaled)


    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    y_pred_linear = (model.predict(X_test_scaled) > 0.5).astype(int)


    model = Ridge()
    model.fit(X_train_scaled, y_train)
    y_pred_ridge = (model.predict(X_test_scaled) > 0.5).astype(int)

    model = Lasso()
    model.fit(X_train_scaled, y_train)
    y_pred_lasso = (model.predict(X_test_scaled) > 0.5).astype(int)

    print(f"N = {n}")
    print('\tLogistic')
    print("\t\tPrecision:", precision(y_test, y_pred_logistic))
    print("\t\tAccuracy:", accuracy(y_test, y_pred_logistic))
    print("\t\tRecall:", recall(y_test, y_pred_logistic), "\n")

    print('\tLinear')
    print("\t\tPrecision:", precision(y_test, y_pred_linear))
    print("\t\tAccuracy:", accuracy(y_test, y_pred_linear))
    print("\t\trecall:", recall(y_test, y_pred_linear), "\n")

    print('\tRidge')
    print("\t\tPrecision:", precision(y_test, y_pred_ridge))
    print("\t\tAccuracy:", accuracy(y_test, y_pred_ridge))
    print("\t\trecall:", recall(y_test, y_pred_ridge), "\n")
    


N = 25
	Logistic
		Precision: 0.8071476736345247
		Accuracy: 0.8036666666666666
		Recall: 0.798 

	Linear
		Precision: 0.8426132145508537
		Accuracy: 0.8076666666666666
		recall: 0.7566666666666667 

	Ridge
		Precision: 0.8426132145508537
		Accuracy: 0.8076666666666666
		recall: 0.7566666666666667 

N = 100
	Logistic
		Precision: 0.9699599465954606
		Accuracy: 0.9693333333333334
		Recall: 0.9686666666666667 

	Linear
		Precision: 0.9907473309608541
		Accuracy: 0.9596666666666667
		recall: 0.928 

	Ridge
		Precision: 0.9907473309608541
		Accuracy: 0.9596666666666667
		recall: 0.928 

N = 500
	Logistic
		Precision: 1.0
		Accuracy: 0.9993333333333333
		Recall: 0.9986666666666667 

	Linear
		Precision: 1.0
		Accuracy: 0.9993333333333333
		recall: 0.9986666666666667 

	Ridge
		Precision: 1.0
		Accuracy: 0.9993333333333333
		recall: 0.9986666666666667 

